# Material Loading

## Imports

In [6]:
import psycopg2
from tabulate import tabulate
#import a folder in the parent directory
import sys
sys.path.append('../')
import dbtools.dbtools as qrs
import dbtools.load as load

## Connection

In [7]:
try:
    # Connect to the PostgreSQL database
    conn = qrs.connect()
    print("Connected to the database")

except (Exception, psycopg2.DatabaseError) as error:
    print(error)

Connected to the database


## Fabrication data

Compulsory information:
- name
- layer_thickness: In mm

Suggested metadata:
- layer_thicnkess: In mm
- fiber_type: nominal
- resin_type: nominal
- prepreg_name: Comercial prepreg name, nominal
- prepreg_type: Extended prepreg properties, text.

In [ ]:
name = 'Prueba 6'

layer_thickness = 0.1

# Define additional metadata
additional_metadata = [
    {'key': 'fiber_type', 'value': 'Hexcel105', 'type': 'nominal'},
    {'key': 'resin_type', 'value': 'Hexcel8552', 'type': 'nominal'},
    {'key': 'prepreg_name', 'value': 'Hexcel8552-105', 'type': 'nominal'},
    {'key': 'prepreg_type', 'value': 'Hexcel8552-105, 0.1mm, 300gsm', 'type': 'text'}
]

# Load material with additional metadata (layer_layout was moved to panel function)
load.load_material(conn, name, layer_thickness, additional_metadata)

Material 'Prueba 6' loaded with ID: 19


19

## Make sure insert was correct

In [9]:
data = qrs.get_data_metadata('materials')

data

,id_material,name_material,layer_thickness_material,density_material,color_material,temperature_resistance_material
0,15,Fibra IM7 Resina M56,0.25 float,NaN,NaN,NaN
1,16,Material Hexcel 0.508 CPT,0.508 float,NaN,NaN,NaN
2,19,Prueba 6,0.1 float,1.5 g/cm3,blue string,250 celsius


In [10]:
conn.close()